In [20]:
import numpy as np

problem = [[1,0,0,0],[1,0,1,0],[0,-1,0,0],[0,0,0,-1],[0,0,1,1]]

In [21]:
def create_formula(problem):
    string = ''
    for row in problem:
        temp = []
        counter = 1
        for element in row:
            if element == 1:
                temp.append('x' + str(counter))
            elif element == -1:
                temp.append('not x' + str(counter))
            counter += 1

        if len(temp) == 1:
            string += '(' + temp[0] + ')' + 'and'
        elif len(temp) > 1:
            string += '(' + ' or '.join(temp) + ')' + 'and'
    return '(' + string[:-3] + ')'

In [22]:
formula1 = create_formula(problem)

In [23]:
formula = """
    ( (x1) and (x1 or x3) and (not x2) and (not x4) and (x3 or x4) )
"""

In [27]:
try:
    import ttg

    print(ttg.Truths(["x1", "x2", "x3", "x4"], [formula1[1:-1]]))
except:
    print("Please import 'truth-table-generator' in order to print the truth table")

+------+------+------+------+--------------------------------------------------------+
|  x1  |  x2  |  x3  |  x4  |  (x1)and(x1 or x3)and(not x2)and(not x4)and(x3 or x4)  |
|------+------+------+------+--------------------------------------------------------|
|  1   |  1   |  1   |  1   |                           0                            |
|  1   |  1   |  1   |  0   |                           0                            |
|  1   |  1   |  0   |  1   |                           0                            |
|  1   |  1   |  0   |  0   |                           0                            |
|  1   |  0   |  1   |  1   |                           0                            |
|  1   |  0   |  1   |  0   |                           1                            |
|  1   |  0   |  0   |  1   |                           0                            |
|  1   |  0   |  0   |  0   |                           0                            |
|  0   |  1   |  1   |  1   |              

In [28]:
from classiq import RegisterUserInput, construct_grover_model

register_size = RegisterUserInput(size=1)

qmod = construct_grover_model(
    num_reps=1,
    expression="(" + formula1 + ")",
    definitions=[
        ("x1", register_size),
        ("x2", register_size),
        ("x3", register_size),
        ("x4", register_size),
    ],
)

In [29]:
from classiq import write_qmod

write_qmod(qmod, "4_sat_grover")

In [30]:
from classiq import QuantumProgram, synthesize, create_model, Preferences, set_preferences

preferences = Preferences(
    backend_service_provider="IBM Quantum", backend_name="ibm_sherbrooke"
)
model = set_preferences(qmod, preferences)
qprog = synthesize(model)

In [31]:
circuit = QuantumProgram.from_qprog(qprog)
circuit.show()

Opening: https://platform.classiq.io/circuit/68c9243d-aa27-4bd8-96e5-fa304f09f1af?version=0.40.0


In [32]:
print(circuit.transpiled_circuit.depth)

626


In [33]:
from classiq import execute, set_quantum_program_execution_preferences
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
)

backend_preferences = ExecutionPreferences(
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR
    )
)

qprog = set_quantum_program_execution_preferences(qprog, backend_preferences)
optimization_result = execute(qprog).result()

In [34]:
res = optimization_result[0].value

In [35]:
res.counts_of_multiple_outputs(("x1", "x2", "x3", "x4"))

{('1', '0', '1', '1'): 82,
 ('0', '1', '0', '0'): 76,
 ('1', '1', '0', '0'): 76,
 ('1', '1', '1', '0'): 81,
 ('1', '1', '0', '1'): 75,
 ('1', '0', '0', '1'): 69,
 ('1', '0', '1', '0'): 934,
 ('0', '1', '1', '1'): 82,
 ('0', '0', '1', '1'): 72,
 ('0', '0', '0', '1'): 74,
 ('0', '1', '1', '0'): 84,
 ('0', '0', '1', '0'): 59,
 ('0', '0', '0', '0'): 71,
 ('0', '1', '0', '1'): 74,
 ('1', '0', '0', '0'): 69,
 ('1', '1', '1', '1'): 70}